In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data= pd.read_csv('/kaggle/input/titanic/train.csv')

test_data= pd.read_csv('/kaggle/input/titanic/test.csv')

raw_train = train_data.copy(deep=True)


# train_data.info()
# cv_set_data.info()

In [3]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
def clean_data(data):
    data.Age.fillna(value=data.Age.mean(), inplace=True)
    data.Fare.fillna(value = data.Fare.mean(),inplace=True)
    data.Fare = data.Fare/data.Fare.mean()
    data.Name = data.Name.apply(lambda x: 'Common' if re.findall('( [a-zA-z]*\.)', x)[0][1:-1] 
                                in ['Miss','Mr','Mrs'] else 'Rare' ) 
    data.Cabin = data.Cabin.apply(lambda x: ord(str(x)[0]))
    data.Name.replace({'Rare': None}, regex=True)
    return data



train_data = clean_data(train_data)
test_data = clean_data(test_data)



In [5]:
# train_data.loc[:,['Name', 'Survived']].groupby(by='Name').sum()
#  train_data.loc[:,['Name','Survived']].value_counts()

def performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_) )
    print('Best Parameters: ' + str(classifier.best_params_))
    print()

In [6]:
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

y= train_data['Survived']

features = ['Pclass', 'Sex', 'SibSp', 'Parch','Age','Fare', 'Name', 'Cabin']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
X_test = X_test.reindex(columns = X.columns,fill_value=0)

# model = SVC(gamma=1e-4, kernel='rbf', C=1e4)

model = SVC(C = 10000.0, coef0 = 0.0, gamma = 0.001, kernel= 'rbf')
model2 = make_pipeline(StandardScaler(),model)
model3 =make_pipeline(StandardScaler(), SVC ( gamma='auto',shrinking=True, tol=1e-10))

cv = cross_val_score(model, X, y, cv =5, scoring ='accuracy')
cv2 = cross_val_score(model2, X, y, cv=5, scoring = 'accuracy')
cv3 = cross_val_score(model3, X, y, cv=5, scoring = 'accuracy')

print(cv.mean() )
print(cv.std())
print()
print(cv2.mean())
print(cv2.std())
print()
print(cv3.mean())
print(cv3.std())
print()


# tuned_parameters = [
#     {'kernel': ['rbf', 'linear'],
#      'C' : [1.0, 2.0, 5.0,10.0, 100.0, 1e4, 1e5, 1e6],
#      'gamma': [1e-5, 1e-4,1e-3, 1e-2, 1e-1, 1, 1e2,1e3 ]
#     }
# ]


# gs_svc = GridSearchCV(model, param_grid = tuned_parameters, cv =5, verbose  = True, n_jobs=-1, scoring = 'accuracy')
# best_fit = gs_svc.fit(X,y)

# performance(best_fit, 'Support Vector Classifier')

model2.fit(X,y)

predictions = model2.predict(X_test)


output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
    
output.to_csv('submission.csv', index=False)


0.8215554579122465
0.022243889787704606

0.8226602222082733
0.020373792681179077

0.8271420500910176
0.018064318689864377



Fitting 5 folds for each of 128 candidates, totalling 640 fits<br>
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.<br>
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.2s<br>
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.9min<br>
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.9min<br>
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed:  7.4min finished<br>
Support Vector Classifier<br>
Best Score: 0.8204255853367648<br>
Best Parameters: {'C': 10000.0, 'gamma': 0.001, 'kernel': 'rbf'}<br>